In [2]:
# Initial setup:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 1

#For fancy plots
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

# import libraries:
import sys, os
import os
import git
path_git = git.Repo('.', search_parent_directories=True).working_tree_dir
os.chdir(path_git); os.sys.path.append('./fr_mcmc/utils/')
from change_of_parameters import omega_CDM_to_luisa
from constants import OMEGA_R_0

sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'../..')))
#path_git = git.Repo('.', search_parent_directories=True).working_tree_dir


from getdist import plots
import getdist
from getdist import plots, MCSamples, loadMCSamples
getdist.chains.print_load_details = False
import scipy
import numpy as np

from matplotlib import pyplot as plt
from scipy.linalg import sqrtm

from seaborn import heatmap
import pandas as pd
import emcee


def F_H(H, params, model):     
    lamb, L, beta, L_bar = params
    if model == 'GILA':
        FH = H**2 \
           + H**8 * L**6     * lamb * np.exp(lamb*(L*H)**2) \
           - H**6 * L_bar**4 * beta * np.exp(-beta*(L_bar*H)**10)
    
    elif model == 'BETA':
        FH = H**2 \
            + H**8 * L**6 * lamb * np.exp(lamb*(L*H)**4) \
            - H**2        * beta * np.exp(-beta*(L_bar*H)**8) 

    return FH

# PPS data

# DESI data

$Q_{DM}$ metric

In [1]:
# Initial setup:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 1

#For fancy plots
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

# import libraries:
import sys, os
import os
import git
path_git = git.Repo('.', search_parent_directories=True).working_tree_dir
os.chdir(path_git); os.sys.path.append('./fr_mcmc/utils/')
from change_of_parameters import omega_CDM_to_luisa
from constants import OMEGA_R_0

sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'../..')))
#path_git = git.Repo('.', search_parent_directories=True).working_tree_dir


from getdist import plots
import getdist
from getdist import plots, MCSamples, loadMCSamples
getdist.chains.print_load_details = False
import scipy
import numpy as np

from matplotlib import pyplot as plt
from scipy.linalg import sqrtm

from seaborn import heatmap
import pandas as pd
import emcee


ModuleNotFoundError: No module named 'matplotlib'

In [5]:
root_dic = '/home/matias/montepython_public/output/'
root_dic = '/home/mleize/montepython_public/output/'

chain_A = getdist.mcsamples.loadMCSamples(file_root=root_dic+'lcdm_cc/2023-09-18_1000000_', no_cache=True,settings={'loglikes':True,'label':'CC','ignore_rows': '0.3'})
chain_B = getdist.mcsamples.loadMCSamples(file_root=root_dic+'lcdm_pps/2023-09-18_1000000_', no_cache=True,settings={'label':'PPS','ignore_rows': '0.3'})
chain_C = getdist.mcsamples.loadMCSamples(file_root=root_dic+'lcdm_planck/2023-09-16_1000000_', no_cache=True,settings={'loglikes':True,'label':'Planck18','ignore_rows': '0.3'})
#chain_D = getdist.mcsamples.loadMCSamples(file_root=root_dic+'old_metrics/lcdm_bao_plus/2024-03-07_1000000_', no_cache=True,settings={'label':'BAO','ignore_rows': '0.3'})


param_names = ['Omega_m','H0']

param_names = ['Omega_m','H0']

mean_A = chain_A.mean(param_names)
cov_A = chain_A.cov(param_names)
mean_B = chain_B.mean(param_names)
cov_B = chain_B.cov(param_names)
mean_C = chain_C.mean(param_names)
cov_C = chain_C.cov(param_names)
#mean_D = chain_D.mean(param_names)
#cov_D = chain_D.cov(param_names)

#Get the gaussian approximations
from tensiometer import gaussian_tension
gaussian_A = gaussian_tension.gaussian_approximation(chain_A)
gaussian_B = gaussian_tension.gaussian_approximation(chain_B)
gaussian_C = gaussian_tension.gaussian_approximation(chain_C)
#aussian_D = gaussian_tension.gaussian_approximation(chain_D)

g = plots.get_subplot_plotter()
#g.triangle_plot([chain_1,chain_2,chain_3,chain_4], params=param_names, legend_labels=['BAO','PPS','Planck18','CC'], filled=True)
g.triangle_plot([gaussian_A,gaussian_B,gaussian_C], params=param_names, legend_labels=['BAO Gaussian','PPS Gaussian','Planck18 Gaussian'], filled=True)


ModuleNotFoundError: No module named 'tensiometer'

In [ ]:
# Get the samples:
# Create the two toy chains and the joint chain:
root_dic = '/home/matias/montepython_public/output/'

chain_1 = getdist.mcsamples.loadMCSamples(file_root=root_dic+'lcdm_cc/2023-09-18_1000000_', no_cache=True,settings={'label':'CC','ignore_rows': '0.3'})
chain_2 = getdist.mcsamples.loadMCSamples(file_root=root_dic+'lcdm_pps/2023-09-18_1000000_', no_cache=True,settings={'label':'PPS','ignore_rows': '0.3'})
chain_12 = getdist.mcsamples.loadMCSamples(file_root=root_dic+'lcdm_pps_cc/2023-09-18_1000000_', no_cache=True,settings={'label':'PPS+CC','ignore_rows': '0.3'})

# import the module:
from tensiometer import mcmc_tension

# set single thread for running on small machines:
mcmc_tension.n_threads = 1

# create the distribution of the parameter differences:
diff_chain = mcmc_tension.parameter_diff_chain(chain_1, chain_2, boost=1)
diff_chain.name_tag = 'MCMC difference'
param_names_diff = ['delta_Omega_m', 'delta_H0']
print(diff_chain.numrows)

# import the module:
from tensiometer import gaussian_tension
param_names = ['Omega_m','H0']

# start calculations:
Q_DM, Q_DM_dofs = gaussian_tension.Q_DM(chain_1,chain_2, param_names=param_names)#, prior_chain=chain_prior_2D)
#Q_DM, Q_DM_dofs = gaussian_tension.Q_DM(chain_1,chain_2)#, param_names=param_names)#, prior_chain=chain_prior_2D)
Q_DM_P = scipy.stats.chi2.cdf(Q_DM, Q_DM_dofs)
#Q_DM_P = scipy.stats.ncx2.cdf(Q_DM, Q_DM_dofs, nc=(mean_1-mean_2)**2)
Q_DM_nsigma = utilities.from_confidence_to_sigma(Q_DM_P)
print(f'Q_DM = {Q_DM:.2f}, dofs = {Q_DM_dofs:2}, PTE = {1-Q_DM_P:.5f}, nsigma = {Q_DM_nsigma:.3f}')


#nwalkers, ndim = reader_lcdm.shape #Number of walkers and parameters
flat_samples_lcdm_pps = samples_lcdm_pps.get_chain(discard=discard, flat=True, thin=thin)
flat_samples_beta_pps = samples_beta_pps.get_chain(discard=discard, flat=True, thin=thin)
flat_samples_gila_pps = samples_gila_pps.get_chain(discard=discard, flat=True, thin=thin)
flat_samples_lcdm_desi = samples_lcdm_desi.get_chain(discard=discard, flat=True, thin=thin)
flat_samples_beta_desi = samples_beta_desi.get_chain(discard=discard, flat=True, thin=thin)
flat_samples_gila_desi = samples_gila_desi.get_chain(discard=discard, flat=True, thin=thin)


L_bar = 0.90
names_LCDM = ['M_{{abs}}','H_0','\omega_m','\Omega_{{m}}^{{LCDM}}','\Omega_{{\\Lambda}}^{{LCDM}}']
labels_LCDM = names_LCDM
names_GILA = ['M_{{abs}}','\\beta','H_0','\omega_m','\Omega_{{m}}^{{GILA-\\beta}}','\Omega_{{m}}^{{LCDM}}','\Omega_{{\\Lambda}}^{{LCDM}}']
labels_GILA = names_GILA

names_GILA_1 = ['\\beta','H_0']
names_LCDM_1 = ['\omega_m','H_0']